### Piano

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus.

* Keyboard image was adapted from [jack-keyboard](http://jack-keyboard.sourceforge.net/).

In [1]:
import logging
import time
import sys
import os

import numpy as np

In [2]:
sys.path.insert(0, os.path.abspath('./..'))

In [3]:
import jupylet.color

from jupylet.app import App
from jupylet.sound import Sample, Synth, sleep, get_oscilloscope_as_image
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

In [4]:
logger = logging.getLogger()

In [5]:
app = App(width=512, height=420)#, log_level=logging.INFO)

In [6]:
a0 = np.zeros((256, 512, 4), 'uint8')

In [7]:
oscilloscope = Sprite(a0, x=256, y=292)

In [8]:
layout = Sprite('images/keyboard.png', x=256, y=82, scale=0.5)

In [9]:
synth = Synth('tri')

In [10]:
synth.set_envelope(duration=0.5, attack=0.0, decay=0.1, sustain=0.7, release=0.5)

In [11]:
keys = app.window.keys

keyboard = {

    keys.Z: 'C',
    keys.S: 'Cs',
    keys.X: 'D',
    keys.D: 'Ds',
    keys.C: 'E',
    keys.V: 'F',
    keys.G: 'Fs',
    keys.B: 'G',
    keys.H: 'Gs',
    keys.N: 'A',
    keys.J: 'As',
    keys.M: 'B',

    keys.Q: 'C5',
    50: 'Cs5',
    keys.W: 'D5',
    51: 'Ds5',
    keys.E: 'E5',
    keys.R: 'F5',
    53: 'Fs5',
    keys.T: 'G5',
    54: 'Gs5',
    keys.Y: 'A5',
    55: 'As5',
    keys.U: 'B5',

    keys.I: 'C6',
    keys.NUMBER_9: 'Cs6',
    keys.O: 'D6',
    48: 'Ds6',
    keys.P: 'E6',
}

In [12]:
state = State(
    
    amp = 1.,
    ms = 50.,
    
    up = False,
    down = False,
    left = False,
    right = False,
)

In [13]:
label0 = Label('amp: %.1f' % state.amp, x=10, y=194)
label1 = Label('span: %.1f ms' % state.ms, x=10, y=174)
label2 = Label('use ← → ↑ ↓ to modify', anchor_x='right', x=app.width - 10, y=174)

In [14]:
pk = {}

In [15]:
@app.event
def key_event(key, action, modifiers):
            
    keys = app.window.keys
    value = action == keys.ACTION_PRESS

    if key == keys.UP:
        state.up = value

    if key == keys.DOWN:
        state.down = value

    if key == keys.RIGHT:
        state.right = value

    if key == keys.LEFT:
        state.left = value
        
    if action == keys.ACTION_PRESS and key in keyboard:
        assert key not in pk
        pk[key] = synth.play_new(note=keyboard[key], duration=20)
        
    if action == keys.ACTION_RELEASE and key in keyboard:
        pk.pop(key).play_release()

In [16]:
@app.run_me_many(1/24)
def modify_oscilloscope(ct, dt):
    
    s = 2 ** dt
    
    if state.up:
        state.amp *= s
        label0.text = 'amp: %.1f' % state.amp

    if state.down:
        state.amp /= s
        label0.text = 'amp: %.1f' % state.amp

    if state.right:
        state.ms *= s
        state.ms = min(256, state.ms)
        label1.text = 'span: %.1f ms' % state.ms

    if state.left:
        state.ms /= s
        label1.text = 'span: %.1f ms' % state.ms

In [17]:
@app.event
def render(ct, dt):
    
    app.window.clear(color='#555')
    
    im, ts, te = get_oscilloscope_as_image(
        1/app.interval,
        ms=state.ms, 
        amp=state.amp, 
        color=255, 
        size=(512, 256)
    )
    
    oscilloscope.image = im    
    oscilloscope.draw()
    
    layout.draw()
    
    label0.draw()
    label1.draw()
    label2.draw()

In [18]:
#app.get_logging_widget()

In [19]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [20]:
#
# Uncomment and run to change keyboard sound and volume:
#
# synth.shape = 'pulse'
# synth.amp = 0.5
#

In [21]:
from jupylet.sound import note

In [22]:
@app.sonic_live_loop(times=10)
async def loop0():
            
        synth.play_new('C2', duration=1.7)
        await sleep(3.)
        
        synth.play_new('E2', duration=1.7)
        await sleep(3.)
            
        synth.play_new('C2', duration=1.7)
        await sleep(6.)
        

In [23]:
@app.sonic_live_loop
async def loop1():
    
    synth.play_new(note.C5, amp=0.5)
    await sleep(1.)

    synth.play_new(note.E5, amp=0.5)
    await sleep(1.)

    synth.play_new(note.G5, amp=0.5)
    await sleep(1.)

In [25]:
#
# Uncomment and run to stop the live loops.
#
# app.stop(loop0)
# app.stop(loop1)
#